In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 500     # number of variables
num_ineq = 500    # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msLinear
model = msLinear(num_var, num_ineq, timelimit=60)

## Heuristic - Round

In [7]:
from src.heuristic import naive_round

In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    try:
        xval_rel, _ = model_rel.solve("gurobi")
        xval, objval = naive_round(xval_rel, model)
        # eval
        params.append(list(b.cpu().numpy()))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model.cal_violation()))
    except:
        params.append(list(b.cpu().numpy()))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())
df.to_csv("result/ln_heur_rnd_500-500.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [09:13<00:00,  5.53s/it]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean  -422.390380         46.050401      1.949912
std      0.482663          3.798879      0.188822
min   -423.742194         39.076604      1.816761
25%   -422.736376         43.475571      1.848241
50%   -422.396551         46.118467      1.863368
75%   -422.107415         48.796764      1.928236
max   -421.219272         54.207219      2.707800
Number of infeasible solution: 100
Number of None values:  0


## Learnable Rounding

In [9]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [10]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [11]:
# set problem
import neuromancer as nm
from src.problem import nmLinear
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmLinear(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [12]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 34830.52
Epoch 1, Validation Loss: -19.78
Epoch 2, Validation Loss: 282.22
Epoch 3, Validation Loss: -52.47
Epoch 4, Validation Loss: -144.48
Epoch 5, Validation Loss: 44.02
Epoch 6, Validation Loss: -176.39
Epoch 7, Validation Loss: -154.40
Epoch 8, Validation Loss: -262.52
Epoch 9, Validation Loss: -208.32
Epoch 10, Validation Loss: -271.59
Epoch 11, Validation Loss: -284.57
Epoch 12, Validation Loss: -262.99
Epoch 13, Validation Loss: -290.55
Epoch 14, Validation Loss: -297.50
Epoch 15, Validation Loss: -296.57
Epoch 16, Validation Loss: -111.98
Epoch 17, Validation Loss: -295.85
Epoch 18, Validation Loss: -283.63
Epoch 19, Validation Loss: -301.17
Epoch 20, Validation Loss: -315.19
Epoch 21, Validation Loss: -234.82
Epoch 22, Validation Loss: -310.63
Epoch 23, Validation Loss: -102.25
Epoch 24, Validation Loss: -237.93
Epoch 25, Validation Loss: -299.33
Epoch 26, Validation Loss: -296.50
Epoch 27, Validation Loss: -299.17
Epoch 28, Validation Loss: -297.53

In [13]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_lr_500-500.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:59<00:00,  1.67it/s]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+02        100.000000    100.000000
mean  -3.441105e+02          0.000325      0.006124
std    5.712979e-13          0.002237      0.001001
min   -3.441105e+02          0.000000      0.002516
25%   -3.441105e+02          0.000000      0.005744
50%   -3.441105e+02          0.000000      0.006184
75%   -3.441105e+02          0.000000      0.006526
max   -3.441105e+02          0.019036      0.008904
Number of infeasible solution: 3


## Learnable Threshold

In [14]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [15]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [16]:
# set problem
import neuromancer as nm
from src.problem import nmLinear
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmLinear(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [17]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 12499.07
Epoch 1, Validation Loss: -36.67
Epoch 2, Validation Loss: -81.73
Epoch 3, Validation Loss: -104.59
Epoch 4, Validation Loss: -140.88
Epoch 5, Validation Loss: -176.22
Epoch 6, Validation Loss: -148.32
Epoch 7, Validation Loss: -244.85
Epoch 8, Validation Loss: -261.28
Epoch 9, Validation Loss: -261.03
Epoch 10, Validation Loss: -264.24
Epoch 11, Validation Loss: -277.37
Epoch 12, Validation Loss: -277.96
Epoch 13, Validation Loss: -285.02
Epoch 14, Validation Loss: -190.92
Epoch 15, Validation Loss: -282.34
Epoch 16, Validation Loss: -292.14
Epoch 17, Validation Loss: -296.57
Epoch 18, Validation Loss: 139.46
Epoch 19, Validation Loss: -307.85
Epoch 20, Validation Loss: -300.77
Epoch 21, Validation Loss: 955.60
Epoch 22, Validation Loss: -301.87
Epoch 23, Validation Loss: -260.38
Epoch 24, Validation Loss: -308.10
Epoch 25, Validation Loss: -84.25
Epoch 26, Validation Loss: 198.12
Epoch 27, Validation Loss: 504.52
Epoch 28, Validation Loss: -235.93
E

In [18]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_lt_500-500.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:04<00:00,  1.56it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean  -356.576696          0.001301      0.006310
std      0.013108          0.006056      0.000912
min   -356.667991          0.000000      0.003310
25%   -356.574833          0.000000      0.006040
50%   -356.574833          0.000000      0.006510
75%   -356.574833          0.000000      0.006721
max   -356.574833          0.045928      0.008204
Number of infeasible solution: 7


### Parametric Learning Then Rounding

In [19]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [20]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [21]:
# set problem
import neuromancer as nm
from src.problem import nmLinear
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmLinear(["b", "x"], num_var, num_ineq, penalty_weight)

In [22]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 12553.04
Epoch 1, Validation Loss: -34.35
Epoch 2, Validation Loss: -90.01
Epoch 3, Validation Loss: -104.82
Epoch 4, Validation Loss: -128.40
Epoch 5, Validation Loss: -34.57
Epoch 6, Validation Loss: -175.91
Epoch 7, Validation Loss: -194.22
Epoch 8, Validation Loss: 134.26
Epoch 9, Validation Loss: -252.55
Epoch 10, Validation Loss: -258.56
Epoch 11, Validation Loss: -268.21
Epoch 12, Validation Loss: -252.87
Epoch 13, Validation Loss: -282.51
Epoch 14, Validation Loss: -250.11
Epoch 15, Validation Loss: -278.07
Epoch 16, Validation Loss: -289.09
Epoch 17, Validation Loss: -265.19
Epoch 18, Validation Loss: -299.57
Epoch 19, Validation Loss: -315.65
Epoch 20, Validation Loss: -308.42
Epoch 21, Validation Loss: -317.41
Epoch 22, Validation Loss: -312.61
Epoch 23, Validation Loss: -323.95
Epoch 24, Validation Loss: -321.52
Epoch 25, Validation Loss: -175.24
Epoch 26, Validation Loss: -320.72
Epoch 27, Validation Loss: -328.57
Epoch 28, Validation Loss: -191.7

In [23]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/ln_pr_500-500.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:08<00:00,  1.46it/s]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+02        100.000000    100.000000
mean  -3.794669e+02          1.066373      0.000864
std    4.570383e-13          0.561258      0.000465
min   -3.794669e+02          0.000000      0.000000
25%   -3.794669e+02          0.648588      0.000998
50%   -3.794669e+02          0.984867      0.001003
75%   -3.794669e+02          1.416090      0.001037
max   -3.794669e+02          2.695822      0.001565
Number of infeasible solution: 99
